In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta import *

In [2]:
builder = SparkSession.builder \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/12/05 21:22:41 WARN Utils: Your hostname, phuc-ASUS resolves to a loopback address: 127.0.1.1; using 192.168.0.101 instead (on interface wlp45s0)
23/12/05 21:22:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/phuc/Practice/DataScience/DSProject/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/phuc/.ivy2/cache
The jars for the packages stored in: /home/phuc/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d28f6206-5104-4a20-aba9-cec8494ed179;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 109ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   | 

In [3]:
career_df = spark.read.format("json").load("/home/phuc/Practice/DataScience/DSProject/data/raw/careerlink/careerlink_2023-12-04T09-33-38+00-00.jsonl")
career_df.show(truncate=False)

+---------------------------------------------------------------------------------+-------------------------------------------------------------------------+----------------------+---------------------------------+--------+-----------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
job_salary = career_df.select("job_experience_requied")
job_salary.show(job_salary.count(), truncate=False)

+----------------------+
|job_experience_requied|
+----------------------+
|2 - 5 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|0 - 1 năm kinh nghiệm |
|0 - 1 năm kinh nghiệm |
|0 - 1 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|0 - 1 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|0 - 1 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|0 - 1 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|0 - 1 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|0 - 1 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|0 - 1 năm kinh nghiệm |
|Cạnh tranh            |
|1 - 2 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|1 - 2 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |
|2 - 5 năm kinh nghiệm |


In [15]:
job_salary.filter(~F.col("job_experience_requied").contains("-")).show(job_salary.count())

+----------------------+
|job_experience_requied|
+----------------------+
|            Cạnh tranh|
|                      |
|  Hơn 10 năm kinh n...|
|                      |
|          Thương lượng|
|          Thương lượng|
|                      |
|          Thương lượng|
|                      |
|          Thương lượng|
|                      |
|  Hơn 10 năm kinh n...|
|                      |
|                      |
|                      |
|  Hơn 10 năm kinh n...|
|                      |
|                      |
|  Hơn 10 năm kinh n...|
|                      |
|             8.1 triệu|
|                      |
|                      |
|                      |
|                      |
|                      |
|                      |
|                      |
|          Thương lượng|
|                      |
|  Hơn 10 năm kinh n...|
|  Hơn 10 năm kinh n...|
|            Cạnh tranh|
|  Hơn 10 năm kinh n...|
|          Thương lượng|
|          Thương lượng|
|  Hơn 10 năm kinh n...|


In [19]:
def get_min_max_yoe(job_experience_required: str):
    if "-" in job_experience_required:
        min_yoe, max_yoe = job_experience_required.split(" - ")[0], job_experience_required.split(" - ")[1].split(" ")[0]
        return {"min": min_yoe, "max": max_yoe}
    elif "Hơn" in job_experience_required:
        return {"min": job_experience_required.split(" ")[1], "max": None}
    return {"min": None, "max": None}

get_min_max_yoe("2 - 5 năm kinh nghiệm")

{'min': '2', 'max': '5'}

In [21]:
from pyspark.sql import DataFrame


@F.udf(
    returnType=T.StructType([
        T.StructField("min", T.IntegerType(), nullable=True),
        T.StructField("max", T.IntegerType(), nullable=True),
    ])
)
def get_min_max_yoe(job_experience_required: str):
    if "năm kinh nghiệm" in job_experience_required:
        if "-" in job_experience_required:
            min_yoe, max_yoe = int(job_experience_required.split(" - ")[0]), int(job_experience_required.split(" - ")[1].split(" ")[0])
            return {"min": min_yoe, "max": max_yoe}
        elif "Hơn" in job_experience_required:
            return {"min": int(job_experience_required.split(" ")[1]), "max": None}
    return {"min": None, "max": None}
    
    
def extract_min_max_yoe(
    df: DataFrame,
    job_experience_reuired_column: str,
    job_yoe_min_column: str,
    job_yoe_max_column: str
) -> DataFrame:
    return (
        df.withColumn("_tmp_job_yoe_required", get_min_max_yoe(F.col(job_experience_reuired_column)))
        .withColumn(job_yoe_min_column, F.col("_tmp_job_yoe_required.min"))
        .withColumn(job_yoe_max_column, F.col("_tmp_job_yoe_required.max"))
        .drop(F.col("_tmp_job_yoe_required"))
    )

job_salary = extract_min_max_yoe(job_salary, "job_experience_requied", "job_yoe_min", "job_yoe_max")
job_salary.show(job_salary.count(), truncate=False)

+----------------------+-----------+-----------+
|job_experience_requied|job_yoe_min|job_yoe_max|
+----------------------+-----------+-----------+
|2 - 5 năm kinh nghiệm |2          |5          |
|1 - 2 năm kinh nghiệm |1          |2          |
|0 - 1 năm kinh nghiệm |0          |1          |
|0 - 1 năm kinh nghiệm |0          |1          |
|0 - 1 năm kinh nghiệm |0          |1          |
|2 - 5 năm kinh nghiệm |2          |5          |
|0 - 1 năm kinh nghiệm |0          |1          |
|2 - 5 năm kinh nghiệm |2          |5          |
|2 - 5 năm kinh nghiệm |2          |5          |
|2 - 5 năm kinh nghiệm |2          |5          |
|0 - 1 năm kinh nghiệm |0          |1          |
|1 - 2 năm kinh nghiệm |1          |2          |
|2 - 5 năm kinh nghiệm |2          |5          |
|2 - 5 năm kinh nghiệm |2          |5          |
|1 - 2 năm kinh nghiệm |1          |2          |
|1 - 2 năm kinh nghiệm |1          |2          |
|1 - 2 năm kinh nghiệm |1          |2          |
|2 - 5 năm kinh nghi